In [8]:
import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype, is_string_dtype, is_datetime64_any_dtype
from IPython.display import display

In [10]:
df=pd.read_csv("heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


# Créons une fonction afin de récupérer les premières informations globales sur notre jeu de données :

Le nombre de colonnes

Le nombre de lignes

Le nombre de doublons

Le nombre de valeurs manquantes

In [17]:
def first_infos(df):
    n_cols = len(df.columns)
    n_rows = len(df)
    n_duplicates = df.duplicated().sum()
    n_na = df.isna().sum().sum()
    return(n_cols, n_rows, n_duplicates, n_na)

# Créons à présent une fonction qui prend en argument un Dataframe :

  - le nombre de variables de type catégorielle, numérique
  
  - texte ou date et qui affiche les infos de first_infos() puis ces dernières

In [18]:
def print_infos(df, n_num, n_cat, n_text, n_date):
    cols, rows, duplicates, na = first_infos(df)
    print('Dataframe :\n\t' + str(rows) + ' Rows \n \t' +
          str(na) + ' Missing values\n\t' +
          str(duplicates) + ' Duplicates\n \t' +
          str(cols) + ' Features\n \t' +
          str(n_cat) + ' Categorical\n \t' +
          str(n_num) + ' Numerical\n \t' +
          str(n_text) + ' Text\n \t' +
          str(n_date) + ' Date')

# Créons une fonction, qui affiche des alertes si une variable contient trop de valeurs manquantes ou contient des données déséquilibrées

In [20]:
def alerts(df):
    # seuils à modifier en fonction des données
    thresh_na = 0.25
    thresh_balance = 0.8
    for col in df.columns:
        if (df[col].count()/len(df)) < thresh_na:
            print('\nThe feature ' + col + ' contains too much missing values!')
        if df[col].value_counts(normalize=True).values[0] > thresh_balance:
            print('\nThe feature ' + col + ' is Imbalanced. Try to fix it !')

# Dans la dernière fonction, à partir du jeu de données, nous créerons un Dataframe contenant des détails supplémentaires concernant le type de données et les valeurs manquantes pour chaque variable, ainsi que des statistiques descriprives.
Les informations globales et les alertes sont affichées en premier, suivi du Dataframe complet.

In [22]:
def quality_check(df):

    # On initialise les variables qui comptabiliseront le nombre de chaque type

    n_text = 0
    n_num = 0
    n_cat = 0
    n_date = 0


    data_qlt_df = pd.DataFrame(index=np.arange(0, len(df.columns)),
                               columns=('column_name', 'col_data_type', 'non_null_values', '%_non_null',
                                        'unique_values_count', 'column_type')
                               )

    for ind, col in enumerate(df.columns):
        # On compte le nombre de valeurs uniques par variable
        col_unique_count = df[col].nunique()

        # On compte le nombre de valeurs non nulles
        non_null = df[col].count()

        # On calcule le pourcentage de valeurs non nulles
        p_non_null = (df[col].count()/len(df))*100

        # On associe à chaque colonne un type en fonction de seuils arbitraires
        if is_datetime64_any_dtype(df[col]):
            col_type = 'Date'
            n_date += 1
        elif is_numeric_dtype(df[col]) & (col_unique_count > 50):
            col_type = 'Numerical'
            n_num += 1
        elif (is_string_dtype(df[col])) & (col_unique_count > 100):
            col_type = 'Text'
            n_text += 1
        else:
            col_type = 'Categorical'
            n_cat += 1

        data_qlt_df.loc[ind] = [col,
                                df[col].dtype,
                                non_null,
                                p_non_null,
                                col_unique_count,
                                col_type
                                ]

    # On utilise la transposé de describe() pour obtenir les statistiques sur les variables numériques
    raw_num_df = df.describe().T.round(2)

    # Puis on merge le résultat avec le reste pour obtenir un Dataframe complet
    data_qlt_df = pd.merge(data_qlt_df, raw_num_df, how='left',
                           left_on='column_name', right_index=True)

    # On affiche les infos globales
    print_infos(df,  n_num, n_cat, n_text, n_date)

    # On affiche les alertes s'il y en a
    alerts(df)

    # afficher le Tableau récapitulatif
    display(data_qlt_df)

# Testons notre fonction sur le jeu de données df

In [25]:
df = pd.read_csv('heart.csv', na_values=[" ", ".", "?", "na"])
quality_check(df)

Dataframe :
	918 Rows 
 	0 Missing values
	0 Duplicates
 	12 Features
 	8 Categorical
 	4 Numerical
 	0 Text
 	0 Date


,column_name,col_data_type,non_null_values,%_non_null,unique_values_count,column_type,count,mean,std,min,25%,50%,75%,max
0,Age,int64,918,100,50,Categorical,918.0,53.51,9.43,28.0,47.00,54.0,60.0,77.0
1,Sex,object,918,100,2,Categorical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ChestPainType,object,918,100,4,Categorical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,RestingBP,int64,918,100,67,Numerical,918.0,132.40,18.51,0.0,120.00,130.0,140.0,200.0
4,Cholesterol,int64,918,100,222,Numerical,918.0,198.80,109.38,0.0,173.25,223.0,267.0,603.0
5,FastingBS,int64,918,100,2,Categorical,918.0,0.23,0.42,0.0,0.00,0.0,0.0,1.0
6,RestingECG,object,918,100,3,Categorical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,MaxHR,int64,918,100,119,Numerical,918.0,136.81,25.46,60.0,120.00,138.0,156.0,202.0
8,ExerciseAngina,object,918,100,2,Categorical,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Oldpeak,float64,918,100,53,Numerical,918.0,0.89,1.07,-2.6,0.00,0.6,1.5,6.2
